In [3]:
%pip install langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn
ERROR: Could not find a version that satisfies the requirement langchain (from versions: none)
ERROR: No matching distribution found for langchain
Note: you may need to restart the kernel to use updated packages.


## LLM Chain

In [4]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [5]:
llm.invoke("how can langsmith help with testing?")

'\nLangsmith is a tool that can assist with testing by providing various features and functionalities. Here are some ways Langsmith can help with testing:\n\n1. Automated Testing: Langsmith provides an API for automating tests, which can be used to create and run test cases automatically. This can save time and reduce the likelihood of human error.\n2. Code Analysis: Langsmith can analyze your codebase and identify potential issues, such as duplicate code, unused variables, or suspicious expressions. This can help you identify areas that need testing.\n3. Test Case Generation: Langsmith can generate test cases based on your codebase and requirements. This can help you create comprehensive test suites and reduce the time and effort required to write tests.\n4. Test Data Generation: Langsmith can also generate test data, such as input values or test cases, based on your requirements. This can help you create more comprehensive test suites and reduce the time and effort required to create

We can also guide it's response with a prompt template. Prompt templates are used to convert raw user input to a better input to the LLM.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

We can now combine these into a simple LLM chain:

In [7]:
chain = prompt | llm

We can now invoke it and ask the same question. It still won't know the answer, but it should respond in a more proper tone for a technical writer!

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that LingSMITH is a powerful tool that can greatly aid in the testing process. Here are some ways in which LingSMITH can help:\n\n1. **Automated Translation**: With LingSMITH's automated translation feature, you can quickly test your software or application with different language versions, without having to manually translate and retest each version. This can significantly reduce the time and resources required for testing.\n2. **Cross-platform Compatibility Testing**: LingSMITH allows you to simulate the user experience of different languages and locales on various platforms, ensuring that your software or application works seamlessly across different environments.\n3. **User Interface (UI) Testing**: LingSMITH's UI testing capabilities allow you to test the look and feel of your application in different languages and locales, including text alignment, formatting, and layout.\n4. **Functional Testing**: With LingSMITH, yo

The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [11]:
chain = prompt | llm | output_parser

We can now invoke it and ask the same question. The answer will now be a string (rather than a ChatMessage).

In [12]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith has a plethora of features that can greatly aid in the testing process. Here are some ways Langsmith can help:\n\n1. Automated Content Generation: Langsmith's AI-powered content generation capabilities can create test cases and test scripts based on your technical documentation, saving you time and effort. This feature allows you to focus on other critical tasks while Langsmith handles the testing process.\n2. Test Case Management: Langsmith provides a robust test case management system that enables you to organize, prioritize, and track your test cases. You can easily search, filter, and run tests based on various criteria, ensuring that all tests are thoroughly executed and documented.\n3. Integration with CI/CD Pipelines: Langsmith seamlessly integrates with popular CI/CD pipelines like Jenkins, Travis CI, CircleCI, and GitLab. This enables you to automate your testing process, m

## Retrieval Chain

In [ ]:
%pip install beautifulsoup4

After that, we can import and use WebBaseLoader.

In [13]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

### index it into a vectorstore. 
This requires a few components, namely 
- an embedding model 
- and a vectorstore.

In [14]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

### Use a simple local vectorstore: FAISS

```
pip install faiss-cpu
```

Then we can build our index:

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

### Create a retrieval chain
This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could run this ourselves by passing in documents directly:



In [17]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by providing a visualization tool for examining and understanding test results.'

However, we want the documents to first come from the retriever we just set up. That way, for a given question we can use the retriever to dynamically select the most relevant documents and pass those in.

In [18]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key

In [19]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


According to the provided context, LangSmith can help with testing in several ways:

1. Creating test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
2. Debugging: LangSmith provides native rendering of chat messages, functions, and retrieve documents, making it easy to identify and root-cause issues in the application's performance.
3. Comparing test runs: LangSmith offers a user-friendly comparison view for test runs, which allows developers to track and diagnose regressions in test scores across multiple revisions of their application.
4. Gathering feedback: LangSmith enables users to attach feedback scores to logged traces and filter on traces that have a specific feedback tag and score, allowing for the gathering of human feedback on the responses produced by the application.
5. Annotating traces: LangSmith supports sending runs to annotation queues, which allow ann

## Conversation Retrieval Chain
Turn this chain into one that can answer follow up questions

We can still use the create_retrieval_chain function, but we need to change two things:

1. The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
2. The final LLM chain should likewise take the whole history into account

Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.

In [20]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

We can test this out by passing in an instance where the user is asking a follow up question.

In [21]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications,

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [22]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

We can now test this out end-to-end:

In [23]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understan

We can see that this gives a coherent answer - we've successfully turned our retrieval chain into a chatbot!

## Agent(只能够用OpenAI这类大模型，local models不够稳定)

NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith
2. A search tool. This will let it easily answer questions that require up to date information.

### Setup a tool for the retriever we just created

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

The search tool that we will use is Tavily. This will require an API key (they have generous free tier). After creating it on their platform, you need to set it as an environment variable:

In [ ]:
export TAVILY_API_KEY=...

If you do not want to set up an API key, you can skip creating this tool.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

We can now create a list of the tools we want to work with:

In [ ]:
tools = [retriever_tool, search]

### Create an agent
Now that we have the tools, we can create an agent to use them. We will go over this pretty quickly

Install langchain hub first
```
pip install langchainhub
```

Now we can use it to get a predefined prompt

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can now invoke the agent and see how it responds! We can ask it questions about LangSmith:

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

We can ask it about the weather:

In [ ]:
agent_executor.invoke({"input": "what is the weather in SF?"})

We can have conversations with it:

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

## Serving with LangServe

Install with:

```bash
pip install "langserve[all]"
```

### Server

To create a server for our application we'll make a serve.py file. This will contain our logic for serving our application. It consists of three things:

1. The definition of our chain that we just built above
2. Our FastAPI app
3. A definition of a route from which to serve the chain, which is done with langserve.add_routes

In [ ]:
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes

# 1. Load Retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]


# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

# We need to add these input/output schemas because the current AgentExecutor
# is lacking in schemas.

class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )


class Output(BaseModel):
    output: str

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

And that's it! If we execute this file. we should see our chain served at `localhost:8000`

In [ ]:
%python serve.py

### Playground

Every LangServe service comes with a simple built-in UI for configuring and invoking the application with streaming output and visibility into intermediate steps. Head to http://localhost:8000/agent/playground/ to try it out! Pass in the same question as before - "how can langsmith help with testing?" - and it should respond same as before.



### Client

Now let's set up a client for programmatically interacting with our service. We can easily do this with the [langserve.RemoteRunnable](/docs/langserve#client). Using this, we can interact with the served chain as if it were running client-side.

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
remote_chain.invoke({
    "input": "how can langsmith help with testing?",
    "chat_history": []  # Providing an empty list as this is the first call
})

## Next steps

We've touched on 
1. how to build an application with LangChain, 
2. how to trace it with LangSmith, 
3. and how to serve it with LangServe. 

There are a lot more features in all three of these than we can cover here. To continue on your journey, we recommend you read the following (in order):

- All of these features are backed by LangChain Expression Language (LCEL) - a way to chain these components together. Check out that documentation to better understand how to create custom chains.
- Model IO covers more details of prompts, LLMs, and output parsers.
- Retrieval covers more details of everything related to retrieval
- Agents covers details of everything related to agents
- Explore common end-to-end use cases and template applications
- Read up on LangSmith, the platform for debugging, testing, monitoring and more
- Learn more about serving your applications with LangServe